In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="PBP Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,1.101639
358,Kyle Lowry,0.589474
419,Moritz Wagner,0.551613
147,Derrick White,0.460328
17,Alperen Sengun,0.422346


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
292,John Konchar,1.556757
583,Xavier Tillman,1.488722
481,Robert Williams III,1.422973
92,Cody Martin,1.400727
232,Ja Morant,1.383784


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
195,Gary Payton II,5.250000
400,Matisse Thybulle,5.089655
128,De'Anthony Melton,4.836522
19,Andre Drummond,4.447059
137,Dejounte Murray,4.186047


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
489,Rudy Gobert,17.833846
244,Jakob Poeltl,16.457143
517,Steven Adams,16.179310
163,Drew Eubanks,15.804878
322,Jusuf Nurkic,15.303273


# The Deadshot Award (presented by Ray Allen/Reggie Miller)

best qualifying 3 point percentage (Basketball-Reference)

1. P.J. Tucker (46.8%)
3. Keldon Johnson (44.1%)
4. Cameron Johnson & Luke Kennard (43.3%)
2. Eric Gordon (42.9%)

# The Stormtrooper Award

worst qualifying 2 point percentage (Basketball-Reference)

1. Devonte' Graham (40.0%)
2. Buddy Hield (41.8%)
3. Nickeil Alexander-Walker (43.1%)
4. Malik Beasley (44.3%)
5. Saddiq Bey (44.4%)

# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
40,Darius Bazley,8
119,Karl-Anthony Towns,7
38,Daniel Gafford,6
84,Ivica Zubac,6
166,Precious Achiuwa,6


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
84,Jaren Jackson Jr.,10
35,Daniel Gafford,9
86,Jarrett Allen,9
8,Andre Drummond,8
138,Myles Turner,8


# The "If He Dies, He Dies" Award (presented by Tom Thibodeau, sponsored by Ivan Drago)

most minutes played per game (Basketball-Reference) (credit to FurryCrew for the idea)

1. Fred VanVleet (38.6)
2. Pascal Siakam (37.5)
3. James Harden (37)
4. Jayson Tatum & Kevin Durant (36.5)

alternatively: most total minutes played (Basketball-Reference) (credit to FrankEMartindale for the idea)

1. Russell Westbrook (1777)
2. Miles Bridges (1758)
3. Jayson Tatum (1751)
4. Mikal Bridges (1734)
5. Julius Randle (1724)

# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,p1,p2,count
189,Clint Capela,Trae Young,47
224,Darius Garland,Jarrett Allen,34
547,Mike Conley,Rudy Gobert,30
305,Dwight Powell,Luka Doncic,27
536,Marcus Smart,Robert Williams III,27


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
100,D.J. Augustin,0.207143,0.008571,0.034286,0.072857,0.380000,0.702857
578,Will Barton,0.207143,0.182857,0.361429,0.008571,0.074286,0.834286
526,Terrence Ross,0.207143,0.120000,0.368571,0.114286,0.031429,0.841429
22,Anfernee Simons,0.491429,0.168571,0.042857,0.011429,0.180000,0.894286
176,Eric Gordon,0.462857,0.011429,0.114286,0.314286,0.125714,1.028571


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME']=='Patrick Beverley'].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
456,Patrick Beverley,0.988571,0.062857,0.874286,0.017143,0.782857,2.725714


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
162,Draymond Green,0.914286,0.822857,0.854286,0.922857,0.945714,4.460000
202,Goga Bitadze,0.780000,0.934286,0.682857,0.968571,0.840000,4.205714
309,Josh Okogie,0.917143,0.694286,0.925714,0.714286,0.882857,4.134286
377,Luka Garza,0.977143,0.877143,0.688571,0.991429,0.597143,4.131429
17,Alperen Sengun,0.974286,0.891429,0.934286,0.957143,0.234286,3.991429


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [15]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
112,Giannis Antetokounmpo,15
166,Jusuf Nurkic,12
137,Jaren Jackson Jr.,10
144,Jerami Grant,10
162,Julius Randle,10
182,Kyle Kuzma,10


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [16]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
29,Clint Capela,12
125,Myles Turner,11
70,JaVale McGee,10
6,Andre Drummond,9
146,Richaun Holmes,9
148,Robert Williams III,9


# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [17]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME_x,count
82,Desmond Bane,6
106,Garrett Temple,6
120,Isaac Okoro,6
293,Tyrese Maxey,6
7,Alec Burks,5
45,Chris Duarte,5
86,Dillon Brooks,5
161,Josh Richardson,5
163,Juan Toscano-Anderson,5
194,Luguentz Dort,5


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (85.5%)
2. Trae Young (84%)
3. Shai Gilgeous-Alexander (83.8%)
4. Chris Paul (82.1%)
4. James Harden (81.4%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Wayne Ellington (95.7%)
2. Garrison Mathews (95.4%)
3. Ben McLemore (95.1%)
4. Davis Bertans (94.9%)
5. Mike Muscala & Maxi Kleber (93.5%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
278,Justin Robinson,0.048
122,Tyus Jones,0.067
103,Aaron Holiday,0.082
262,Trae Young,0.093
229,Tyrese Haliburton,0.094
297,Immanuel Quickley,0.094


In [19]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
28,Carmelo Anthony,0.305


alternatively: restricting to players > 6 foot 6 inches in height

In [20]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
47,Furkan Korkmaz,0.111
18,Cedi Osman,0.138
158,Ziaire Williams,0.150
41,Duncan Robinson,0.152
81,Jordan Nwora,0.152


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [21]:
reb_w_gp_qualify.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
186,Jock Landale,0.597
74,Mitchell Robinson,0.587
327,Luka Garza,0.570
172,Jakob Poeltl,0.566
303,Justin Champagnie,0.563


alternatively: restricting to players < 6 foot 7 inches in height

In [22]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
108,Justin Champagnie,0.563
174,Thanasis Antetokounmpo,0.442
116,Kenrich Williams,0.399
141,Moses Moody,0.394
119,Kenyon Martin Jr.,0.385


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as initial baseline for position groupings**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)